## SpaRTaNS Tutorial: Double Chamber Flow
#### Last updated: 04/30/2022

## Bounce Properties

- We went through all that trouble above to get a mesh with a one-to-one nodal and triangle correspondence between the periodic surfaces
  - as-well as automatically identified boundary groupings
- Here, we use these to obtain the triangle mapping requested by SpaRTaNS
  - This has the form {outgoing_triangle_index, outgoing_triangle_vertex_index, incoming_triangle_index, incoming_triangle_vertex_index}
    - for each "bounce tensor"

In [ ]:
triangles["SO2"]["double-chamber"] = Thread[mesh["SO2"]["double-chamber"]["BoundaryElements"][[1]]] /. TriangleElement[a__] :> List[a];
vertices["SO2"]["double-chamber"] = mesh["SO2"]["double-chamber"]["Coordinates"];
Do[positions["SO2"]["double-chamber"][j] = Flatten[Position[triangles["SO2"]["double-chamber"], {{t1_, t2_, t3_}, i_} /; i == j]], 
    {j, Length[mesh["SO2"]["double-chamber"]["BoundaryElementMarkerUnion"]]}]

- We will use two bounce tensors
  - Identity scattering across the periodic surfaces of the channel
    - i.e. carriers from surface 4 pass straight through to surface 8, and vice-versa

In [ ]:
bottomTriangles["SO2"]["double-chamber"] = 
  Extract[vertices["SO2"]["double-chamber"], 
   List /@ triangles["SO2"]["double-chamber"][[
     positions["SO2"]["double-chamber"][4], 1]]];
bottomTrianglesOrdering["SO2"]["double-chamber"] = 
  Ordering /@ bottomTriangles["SO2"]["double-chamber"];
bottomTrianglesMapped["SO2"]["double-chamber"] = 
  MapAt[-# &, bottomTriangles["SO2"]["double-chamber"], {All, All, 2}];
topTriangles["SO2"]["double-chamber"] = 
  Extract[vertices["SO2"]["double-chamber"], 
   List /@ triangles["SO2"]["double-chamber"][[
     positions["SO2"]["double-chamber"][8], 1]]];
topTrianglesOrdering["SO2"]["double-chamber"] = 
  Ordering /@ topTriangles["SO2"]["double-chamber"];

bottomToTopMapping["SO2"]["double-chamber"] = 
  Position[Sort /@ topTriangles["SO2"]["double-chamber"], #][[1, 
     1]] & /@ (Sort /@ bottomTrianglesMapped["SO2"]["double-chamber"]);
bottomToTopMapping["SO2"]["double-chamber"] = 
  Join @@ MapIndexed[
    Table[{positions["SO2"]["double-chamber"][4][[First[#2]]], i[[1]],
        positions["SO2"]["double-chamber"][8][[#1]], i[[2]]}, {i, 
       Thread[{bottomTrianglesOrdering["SO2"]["double-chamber"][[
          First[#2]]], 
         topTrianglesOrdering["SO2"]["double-chamber"][[#1]]}]}] &, 
    bottomToTopMapping["SO2"]["double-chamber"]];
periodicMapping["SO2"]["double-chamber"] = 
  Join[bottomToTopMapping["SO2"]["double-chamber"], 
   Join @@ Reverse[Partition[#, 2]] & /@ 
    bottomToTopMapping["SO2"]["double-chamber"]];

  - diffuse scattering across all other surfaces

In [ ]:
diffusiveMapping["SO2"]["double-chamber"] = 
  Flatten[Table[{positions["SO2"]["double-chamber"][i][[n]], t, 
     positions["SO2"]["double-chamber"][i][[n]], 
     t}, {i, {1, 2, 3, 5, 6, 7, 9, 10}}, {n, 
     Length[positions["SO2"]["double-chamber"][i]]}, {t, 3}], 2];

  - Note: Spartans unlike Mathematica uses 0-based indexing

In [ ]:
triangleMappings["SO2"][
   "double-chamber"] = {diffusiveMapping["SO2"]["double-chamber"], 
    periodicMapping["SO2"]["double-chamber"]} - 1;

- We then define our bounce tensors

In [ ]:
bounceTensors["SO2"]["double-chamber"] = {
   ConstantArray[2./numberOfStates, {numberOfStates, numberOfStates}],
   N[IdentityMatrix[numberOfStates]]
   };